# Ver 1: Valid Output+Random Parameters

In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import numpy as np
from typing import Dict, Tuple, List
import helper
import matplotlib.pyplot as plt

In [2]:
import torch
from torch.utils.data import Dataset
import numpy as np

class LinearModelDataset(Dataset):
    def __init__(self, num_samples: int, nr: int, nc: int, dt: float, F: float):
        self.num_samples = num_samples
        self.nr = nr
        self.nc = nc
        self.dt = dt
        self.F = F
        self.samples = self._generate_samples()

    def _generate_gaussian_field(self, n, nrv, ncv):
        """
        Generate a Gaussian field composed of n Gaussian functions.
        """
        mux = np.random.choice(ncv, n)
        muy = np.random.choice(range(2, nrv - 2), n)
        sigmax = np.random.uniform(1, ncv/4, n)
        sigmay = np.random.uniform(1, nrv/4, n)

        v = np.zeros((nrv, ncv))
        for i in range(n):
            for x in range(ncv):
                for y in range(nrv):
                    # Create three copies for pseudo-periodic field
                    gauss = np.exp(-((x-mux[i])**2/(2*sigmax[i]**2) + (y-muy[i])**2/(2*sigmay[i]**2)))
                    gauss += np.exp(-((x-(mux[i]-ncv))**2/(2*sigmax[i]**2) + (y-muy[i])**2/(2*sigmay[i]**2)))
                    gauss += np.exp(-((x-(mux[i]+ncv))**2/(2*sigmax[i]**2) + (y-muy[i])**2/(2*sigmay[i]**2)))
                    v[y,x] += gauss
        return v

    def _generate_circular_field(self, v):
        """
        Generate a circular field from gradient of input field.
        """
        grad_v_y, grad_v_x = np.gradient(v)
        return -grad_v_y, grad_v_x

    def _generate_params(self):
        """
        Generate model parameters similar to generate_world function.
        """
        # Base grid parameters
        DX_C = torch.ones(self.nr, self.nc + 1)
        DY_C = torch.ones(self.nr + 1, self.nc)
        DX_G = torch.ones(self.nr + 1, self.nc)
        DY_G = torch.ones(self.nr, self.nc + 1)
        RAC = torch.ones(self.nr, self.nc)

        # Generate random diffusivities (must be positive)
        KX = torch.abs(torch.rand(self.nr, self.nc + 1))
        KY = torch.abs(torch.rand(self.nr + 1, self.nc))

        # Generate velocities using Gaussian field approach
        num_gauss = 16  # Number of Gaussian functions for velocity field
        gauss = self._generate_gaussian_field(num_gauss, self.nr + 1, self.nc + 1)
        VX_np, VY_np = self._generate_circular_field(gauss)
        
        # Convert velocities to PyTorch and scale
        VX = torch.from_numpy(100 * VX_np[:-1, :]).float()
        VY = torch.from_numpy(100 * VY_np[:, :-1]).float()

        # Create random forcing term
        f = torch.randn(self.nr * self.nc)

        return {
            'KX': KX,
            'KY': KY,
            'DX_C': DX_C,
            'DY_C': DY_C,
            'DX_G': DX_G,
            'DY_G': DY_G,
            'VX': VX,
            'VY': VY,
            'RAC': RAC,
            'f': f,
        }

    def _generate_samples(self) -> list:
        """
        Generate multiple samples with parameters.
        """
        samples = []
        for _ in range(self.num_samples):
            params = self._generate_params()
            
            x_t = torch.randn(self.nr * self.nc)
            lambda_t_plus_1 = torch.randn(self.nr * self.nc)
            dx_dt_lambda = self._compute_dx_dt_lambda(params, x_t, lambda_t_plus_1)
            
            samples.append((params, x_t, lambda_t_plus_1, dx_dt_lambda))
        
        return samples

    def _compute_dx_dt_lambda(self, params, x_t, lambda_t_plus_1):
        """
        Compute dx(t+1)/dx(t) * lambda(t+1) for the linear model.
        """
        # Convert PyTorch tensors to numpy arrays for helper function
        np_params = {
            key: tensor.cpu().detach().numpy() if torch.is_tensor(tensor) else tensor
            for key, tensor in params.items()
        }
        
        # Get model matrix M using helper function
        M = helper.make_M_2d_diffusion_advection_forcing(
            nr=self.nr,
            nc=self.nc,
            dt=self.dt,
            KX=np_params['KX'],
            KY=np_params['KY'],
            DX_C=np_params['DX_C'],
            DY_C=np_params['DY_C'], 
            DX_G=np_params['DX_G'],
            DY_G=np_params['DY_G'],
            VX=np_params['VX'],
            VY=np_params['VY'],
            RAC=np_params['RAC'],
            F=self.F,
            cyclic_east_west=True,
            cyclic_north_south=False,
            M_is_sparse=False
        )
        
        # Convert to numpy arrays and do matrix multiplication
        lambda_np = lambda_t_plus_1.cpu().detach().numpy()
        result_np = M.T @ lambda_np
        
        # Convert back to PyTorch tensor
        return torch.from_numpy(result_np).float()

    def __len__(self) -> int:
        return self.num_samples

    def __getitem__(self, idx: int):
        return self.samples[idx]

class LinearModelNet(torch.nn.Module):
    def __init__(self, nr: int, nc: int):
        super().__init__()
        self.nr = nr
        self.nc = nc
        
        self.state_size = nr * nc
        self.param_size = (nr * (nc + 1) +
                           (nr + 1) * nc +
                           nr * (nc + 1) +
                           (nr + 1) * nc +
                           (nr + 1) * nc +
                           nr * (nc + 1) +
                           nr * (nc + 1) +
                           (nr + 1) * nc +
                           nr * nc +
                           nr * nc)
        
        self.input_size = self.state_size * 2 + self.param_size
        self.output_size = nr * nc
        
        print(f"Input size: {self.input_size}")
        
        # Adjust the architecture to handle the correct input size
        self.fc1 = torch.nn.Linear(self.input_size, 2048)
        self.fc2 = torch.nn.Linear(2048, 1024)
        self.fc3 = torch.nn.Linear(1024, 512)
        self.fc4 = torch.nn.Linear(512, self.output_size)
        
        self.activation = torch.nn.ReLU()
        
    def forward(self, x: torch.Tensor, lambda_t_plus_1: torch.Tensor, params: Dict[str, torch.Tensor]) -> torch.Tensor:
        batch_size = x.size(0) 
        
        # Ensure all tensors have the correct batch size
        x = x.view(batch_size, -1)
        lambda_t_plus_1 = lambda_t_plus_1.view(batch_size, -1)
        
        # Flatten the params dictionary
        params_flat = torch.cat([param.view(batch_size, -1) for param in params.values()], dim=1)
        
        # Concatenate all inputs
        x = torch.cat([x, lambda_t_plus_1, params_flat], dim=1)
        
        # Add some debugging print statements
        #print(f"x shape: {x.shape}")
        #print(f"Input size: {self.input_size}")
        
        x = self.activation(self.fc1(x))
        x = self.activation(self.fc2(x))
        x = self.activation(self.fc3(x))
        x = self.fc4(x)
        
        return x

def train_model(model: torch.nn.Module, dataloader: DataLoader, num_epochs: int, learning_rate: float):
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()
    
    for epoch in range(num_epochs):
        total_loss = 0
        for params, x_t, lambda_t_plus_1, dx_dt_lambda in dataloader:
            optimizer.zero_grad()
            output = model(x_t, lambda_t_plus_1, params)
            loss = criterion(output, dx_dt_lambda)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss/len(dataloader):.4f}")



In [ ]:
import torch
import numpy as np
from typing import Dict, Tuple
import os
import h5py
from tqdm import tqdm

def generate_and_save_dataset(
    filename: str,
    num_train_samples: int,
    num_test_samples: int,
    nr: int,
    nc: int,
    dt: float,
    F: float
) -> None:
    """
    Generate training and testing datasets and save them to an HDF5 file.
    
    Args:
        filename: Name of the file to save the data to
        num_train_samples: Number of training samples to generate
        num_test_samples: Number of testing samples to generate
        nr, nc: Grid dimensions
        dt: Time step
        F: Forcing parameter
    """
    # Create dataset instances
    train_dataset = LinearModelDataset(num_train_samples, nr, nc, dt, F)
    test_dataset = LinearModelDataset(num_test_samples, nr, nc, dt, F)
    
    with h5py.File(filename, 'w') as f:
        # Create train and test groups
        train_group = f.create_group('train')
        test_group = f.create_group('test')
        
        # Save metadata
        f.attrs['nr'] = nr
        f.attrs['nc'] = nc
        f.attrs['dt'] = dt
        f.attrs['F'] = F
        
        # Helper function to save a single dataset
        def save_dataset(group, dataset, desc):
            for i, (params, x_t, lambda_t_plus_1, dx_dt_lambda) in enumerate(tqdm(dataset, desc=desc)):
                sample_group = group.create_group(f'sample_{i}')
                
                # Save parameters
                params_group = sample_group.create_group('params')
                for key, value in params.items():
                    params_group.create_dataset(key, data=value.numpy())
                
                # Save state and outputs
                sample_group.create_dataset('x_t', data=x_t.numpy())
                sample_group.create_dataset('lambda_t_plus_1', data=lambda_t_plus_1.numpy())
                sample_group.create_dataset('dx_dt_lambda', data=dx_dt_lambda.numpy())
        
        # Save training and testing datasets
        save_dataset(train_group, train_dataset, "Saving training data")
        save_dataset(test_group, test_dataset, "Saving testing data")

def load_dataset(filename: str) -> Tuple[Dict, Dict]:
    """
    Load training and testing datasets from an HDF5 file.
    
    Args:
        filename: Name of the file to load the data from
        
    Returns:
        Tuple of dictionaries containing training and testing data
    """
    train_data = []
    test_data = []
    
    with h5py.File(filename, 'r') as f:
        # Load metadata
        metadata = {
            'nr': f.attrs['nr'],
            'nc': f.attrs['nc'],
            'dt': f.attrs['dt'],
            'F': f.attrs['F']
        }
        
        # Helper function to load a single dataset
        def load_dataset(group):
            data = []
            for sample_name in tqdm(group.keys(), desc=f"Loading {group.name} data"):
                sample = group[sample_name]
                
                # Load parameters
                params = {
                    key: torch.from_numpy(value[:]).float()
                    for key, value in sample['params'].items()
                }
                
                # Load state and outputs
                x_t = torch.from_numpy(sample['x_t'][:]).float()
                lambda_t_plus_1 = torch.from_numpy(sample['lambda_t_plus_1'][:]).float()
                dx_dt_lambda = torch.from_numpy(sample['dx_dt_lambda'][:]).float()
                
                data.append((params, x_t, lambda_t_plus_1, dx_dt_lambda))
            return data
        
        # Load training and testing datasets
        train_data = load_dataset(f['train'])
        test_data = load_dataset(f['test'])
    
    return {
        'train': train_data,
        'metadata': metadata
    }, {
        'test': test_data,
        'metadata': metadata
    }

class SavedDataset(torch.utils.data.Dataset):
    """Dataset class for loading pre-saved data"""
    def __init__(self, data):
        self.data = data
    
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return self.data[idx]
    
nr,nc=10,10

"""
Time to generate the dataset and train the model
: 3s

generate_and_save_dataset(
    filename='linear_model_data.h5',
    num_train_samples=180,
    num_test_samples=20,
    nr=10,
    nc=10,
    dt=0.1,
    F=1.0
)"""
"""
generate_and_save_dataset(
    filename='linear_model_data.h5',
    num_train_samples=180000,
    num_test_samples=20000,
    nr=10,
    nc=10,
    dt=0.1,
    F=1.0
)
"""

Saving testing data: 100%|██████████| 20000/20000 [00:43<00:00, 454.99it/s]


### To-do:

- Set things up so we can save data
- Train with more data to see if it can learn effectively

- Try covariance (maybe test model on OOD covariance?)
- Try data from a model simulated over time
- See how well that does on backprop

In [12]:
import torch
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

def train_and_evaluate_model(
    model: torch.nn.Module,
    train_dataloader: DataLoader,
    test_dataloader: DataLoader,
    num_epochs: int,
    learning_rate: float
):
    """
    Train the model and evaluate its performance with visualization
    """
    # Training setup
    optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
    criterion = torch.nn.MSELoss()
    
    # Lists to store losses for plotting
    train_losses = []
    test_losses = []
    
    print("Training started...")
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        total_train_loss = 0
        for params, x_t, lambda_t_plus_1, dx_dt_lambda in train_dataloader:
            optimizer.zero_grad()
            output = model(x_t, lambda_t_plus_1, params)
            loss = criterion(output, dx_dt_lambda)
            loss.backward()
            optimizer.step()
            total_train_loss += loss.item()
        
        # Evaluation phase
        model.eval()
        total_test_loss = 0
        with torch.no_grad():
            for params, x_t, lambda_t_plus_1, dx_dt_lambda in test_dataloader:
                output = model(x_t, lambda_t_plus_1, params)
                test_loss = criterion(output, dx_dt_lambda)
                total_test_loss += test_loss.item()
        
        # Calculate average losses
        avg_train_loss = total_train_loss / len(train_dataloader)
        avg_test_loss = total_test_loss / len(test_dataloader)
        
        # Store losses for plotting
        train_losses.append(avg_train_loss)
        test_losses.append(avg_test_loss)
        
        if (epoch + 1) % 10 == 0:  # Print every 10 epochs
            print(f"Epoch {epoch+1}/{num_epochs}")
            print(f"Training Loss: {avg_train_loss:.4f}")
            print(f"Test Loss: {avg_test_loss:.4f}")
    
    print("Training complete!")
    
    # Plotting
    plt.figure(figsize=(10, 6))
    epochs = range(1, num_epochs + 1)
    plt.plot(epochs, train_losses, 'b-', label='Training Loss')
    plt.plot(epochs, test_losses, 'r-', label='Test Loss')
    plt.title('Training and Testing Loss Over Time')
    plt.xlabel('Epoch')
    plt.ylabel('Loss (MSE)')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    # Evaluate on test examples
    model.eval()
    test_params, test_x, test_lambda, test_true = next(iter(test_dataloader))
    
    # Get model prediction for first test example
    with torch.no_grad():
        predicted = model(test_x[0:1], test_lambda[0:1], 
                        {k: v[0:1] for k, v in test_params.items()})
    
    # Remove batch dimension
    predicted = predicted.squeeze(0)
    test_true = test_true[0]  # Get first example's true values
    
    # Print comparison
    print(f"\nComparison of first 10 values (from test set):")
    print(f"{'Index':<6} {'True Value':>12} {'Predicted':>12} {'Difference':>12}")
    print("-" * 44)
    for i in range(10):
        print(f"{i:<6} {test_true[i]:>12.6f} {predicted[i]:>12.6f} {(test_true[i] - predicted[i]):>12.6f}")
    
    # Print overall statistics
    print(f"\nTest Set Statistics:")
    print(f"Mean Absolute Error: {torch.abs(test_true - predicted).mean():.6f}")
    print(f"Root Mean Square Error: {torch.sqrt(torch.mean((test_true - predicted)**2)):.6f}")
    print(f"True vector norm: {torch.norm(test_true):.6f}")
    print(f"Predicted vector norm: {torch.norm(predicted):.6f}")
    
    # Plot true vs predicted values
    plt.figure(figsize=(12, 6))
    plt.plot(test_true.numpy(), label='True Values', marker='o')
    plt.plot(predicted.numpy(), label='Predicted Values', marker='x')
    plt.title('True vs Predicted Values (Test Set)')
    plt.xlabel('Index')
    plt.ylabel('Value')
    plt.legend()
    plt.grid(True)
    plt.show()
    
    return model, train_losses, test_losses

In [ ]:
# Create and train the model
nr,nc=10,10
train_data, test_data = load_dataset('linear_model_data.h5')

# Create datasets and dataloaders
train_dataset = SavedDataset(train_data['train'])
test_dataset = SavedDataset(test_data['test'])

train_dataloader = DataLoader(train_dataset, batch_size=320, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=320, shuffle=False)

model = LinearModelNet(nr, nc)
trained_model, train_losses, test_losses = train_and_evaluate_model(
    model=model,
    train_dataloader=train_dataloader,
    test_dataloader=test_dataloader,
    num_epochs=600,
    learning_rate=0.001
)

Loading /test data: 100%|██████████| 20000/20000 [01:36<00:00, 208.14it/s]


Input size: 1280
Training started...
Epoch 10/600
Training Loss: 169.3921
Test Loss: 169.7578
Epoch 20/600
Training Loss: 168.7892
Test Loss: 169.2647
Epoch 30/600
Training Loss: 166.2119
Test Loss: 166.7372
Epoch 40/600
Training Loss: 165.8158
Test Loss: 166.4536
Epoch 50/600
Training Loss: 165.6289
Test Loss: 166.3011
Epoch 60/600
Training Loss: 165.4453
Test Loss: 165.9948
Epoch 70/600
Training Loss: 165.3685
Test Loss: 165.9264
Epoch 80/600
Training Loss: 165.3003
Test Loss: 166.0233
Epoch 90/600
Training Loss: 165.2725
Test Loss: 165.9051
Epoch 100/600
Training Loss: 165.2612
Test Loss: 166.0332
Epoch 110/600
Training Loss: 165.2113
Test Loss: 165.9668
Epoch 120/600
Training Loss: 165.2123
Test Loss: 165.9913
Epoch 130/600
Training Loss: 165.1555
Test Loss: 165.9623
Epoch 140/600
Training Loss: 165.1470
Test Loss: 166.1304
Epoch 150/600
Training Loss: 165.1412
Test Loss: 166.2851
Epoch 160/600
Training Loss: 165.1010
Test Loss: 166.1456
Epoch 170/600
Training Loss: 165.0623
Test L